In [4]:
import os
import numpy as np
import itertools
import trimesh
import math
import k3d
from time import sleep
from tqdm import tqdm_notebook as tqdm

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from geometry.model import Model, combine_observations, get_mesh
from geometry.utils.visualisation import illustrate_points, illustrate_mesh
from rl.environment import *

from geometry.voxel_grid import *
builder = VoxelGridBuilder(32)


# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym
# jupyter nbextension install --py --sys-prefix k3d
# jupyter nbextension enable --py --sys-prefix k3d

In [12]:
class Environment(gym.Env):
    def __init__(self,
                 models_path=None,
                 model_path=None,
                 number_of_view_points=100,
                 similarity_threshold=0.95,
                 image_size=512,
                 illustrate=False):
        super().__init__()

        self.model_path = model_path
        self.models_path = models_path
        self.number_of_view_points = number_of_view_points
        self.image_size = image_size
        self.illustrate = illustrate

        self.action_space = spaces.Discrete(number_of_view_points)
        self.observation_space = spaces.Dict({
            'depth_map': spaces.Box(-np.inf, np.inf, (image_size, image_size), dtype=np.float32),
        })

        self._similarity_threshold = similarity_threshold
        self._reconstruction_depth = 10

        self.model = None
        self.plot = None

    def reset(self, init_action=None):
        """
        Reset the environment for new episode.
        Randomly (or not) generate CAD model for this episode.
        """
        if self.model_path is not None:
            model_path = self.model_path
        elif self.models_path is not None:
            model_path = os.path.join(self.models_path,
                                      random.sample(os.listdir(self.models_path), 1)[0])
        self.model = Model(model_path, resolution_image=self.image_size)
        self.model.generate_view_points(self.number_of_view_points)
        
        if self.illustrate:
            self.model.illustrate().display()

            self.plot = k3d.plot()
            self.plot.display()
        
        if init_action is None:
            init_action = self.action_space.sample()
        observation = self.model.get_observation(init_action)
        return observation, init_action

    def step(self, action):
        """
        Get new observation from current position (action), count step reward, decide whether to stop.
        Args:
            action: int
        return: 
            next_state: List[List[List[int, int, int]]]
            reward: float
            done: bool
            info: Tuple
        """
        assert self.action_space.contains(action)
        
        view_point = self.model.view_points[action]
        new_view_point = add_noise_to_vp(view_point, self.model)
        observation = self.model.get_observation_from_point(view_point)

        reward = self.step_reward(observation)
        done = reward >= self._similarity_threshold

        return observation, reward, done, {}
    
    def render(self, action, observation, plot=None):
        if plot is None:
            plot = self.plot
            
        plot = illustrate_points(
           [self.model.get_point(action)], size=0.5, plot=plot)
        
        plot = observation.illustrate(plot, size=0.03)
        return plot
    
    def step_reward(self, observation):
        return self.model.observation_similarity(observation)
    
        if self.illustrate:
            illustrate_mesh(vertices, faces).display()
        return reward
        
    def _get_mesh(self, observation):
        faces, vertices = poisson_reconstruction(observation.points,
                                                 observation.normals,
                                                 depth=self._reconstruction_depth)
        return vertices, faces

    def final_reward(self, observation):
        vertices, faces = self._get_mesh(observation)
        reward = self.model.surface_similarity(vertices, faces)
        return reward



def create_env(model_path=None, number_of_view_points=100):    
    env = Environment(model_path=model_path,
                      image_size=1024,
                      number_of_view_points=number_of_view_points)

    env = MeshReconstructionWrapper(env, reconstruction_depth=8, final_depth=10, scale_factor=8,
                                    do_step_reconstruction=True)
    env = VoxelGridWrapper(env, grid_size=32)
    env = CombiningObservationsWrapper(env)
    env = VoxelWrapper(env, occlusion_reward=True)
    return env

### Create your function

In [13]:
from geometry.model import ViewPoint

def add_noise_to_vp(view_point, model):
    # YOUR CODE IS HERE
    # use model.bounds / 32
    point = view_point.point
    phi = view_point.phi
    theta = view_point.theta
    return ViewPoint(point, phi, theta)

In [17]:
from tqdm import tqdm_notebook as tqdm

models_paths = [
    "./data/00070090_73b2f35a88394199b6fd1ab8_003.obj",
    "./data/00060056_5f75fa75edda41c2a78510a7_000.obj",
    "./data/val/00010116_a35af5bb7a7a42fe9539f3b7_008.obj",
    "./data/val/00010152_13f1e6a043494993aa8494b7_000.obj",
    "./data/val/00010155_ccef4063b69f428e91b498c9_000.obj",
    "./data/val/00010159_ccef4063b69f428e91b498c9_004.obj",
    "./data/val/00010169_859b5c4a8376437fa24c285b_001.obj",
    "./data/val/00010174_30f2ce809de74ac596f8a219_000.obj",
    "./data/val/00020151_b27a1602d1d44a3d89140ce4_051.obj",
    "./data/val/00020228_f7a8b12633374114bcbd6244_000.obj"]


view_point_idxs = [
    [63, 29, 54],
    [35, 65, 37],
    [8, 85],
    [93, 18],
    [94, 1, 93],
    [3, 88],
    [15, 79, 77],
    [26, 77],
    [0, 93, 94, 38],
    [1, 68]
]

for model_path, vp_idxs in zip(models_paths, view_point_idxs):
    
    env = create_env(model_path, number_of_view_points=100)

    state, action = env.reset(init_action=vp_idxs[0])
    states = [state]
    model = env.model
    
    for action in vp_idxs[1:]:
        state, reward, done, info = env.step(action)
        states.append(state)

    voxels_after_nbv = np.count_nonzero(states[-1] == 2)
    
    combined_observation = None
    for view_point_idx in vp_idxs:
        view_point = model.view_points[view_point_idx]
        new_view_point = add_noise_to_vp(view_point, model)
        observation = model.get_observation_from_point(view_point)

        if combined_observation is None:
            combined_observation = observation
        else:
            combined_observation += observation

    reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)
    loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)

    print(model_path)
    print("View points: ", vp_idxs)
    print("Voxels after NBV: ", voxels_after_nbv)
    print("Hausdorff: ", loss)
    
    break

./data/00070090_73b2f35a88394199b6fd1ab8_003.obj
View points:  [63, 29, 54]
Voxels after NBV:  6430
Hausdorff:  0.3345277645033892


### Illustrations

In [18]:
# MODEL_PATH = "./data/00070090_73b2f35a88394199b6fd1ab8_003.obj"

# model = Model(model_path=MODEL_PATH)
# model.generate_view_points(10)

model.illustrate()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

In [19]:
illustrate_mesh(reconstructed_vertices, reconstructed_faces)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

In [20]:
bounds = np.array([np.min(reconstructed_vertices, axis=0), np.max(reconstructed_vertices, axis=0)])
mesh_grid_rec = builder.build(reconstructed_vertices, bounds)



plot = k3d.plot()
plt_voxels = k3d.voxels(mesh_grid_rec.grid() * 3)
plot += plt_voxels
plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

In [21]:
env.render(action, state)

Output()

In [25]:
vp_idxs

[63, 29, 54]

In [28]:
plot = illustrate_voxels(states[-1] * 3)
illustrate_points(vp_idxs[:1], size=5, plot=plot)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

In [26]:
env.model.illustrate()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…